In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch
from torch.utils.data import Dataset, DataLoader
import pickle
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score,accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,roc_auc_score,ConfusionMatrixDisplay,precision_score,recall_score

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
main_dir = 'drive/MyDrive/23Spring-MLMA/continuous1/'
os.listdir(main_dir)

['ReadMe',
 '150word_label.xlsx',
 '50character_label.xlsx',
 'common',
 '004',
 '001',
 '006',
 '005',
 '003',
 '002']

In [ ]:
patient = ["001","002","003","004","005","006"]
test_split = [["01","02"],["03","04"],["05","06"],["07","08"],["09","10"]]
val_split = ['03','01',"04","05","06"]

In [ ]:
import pandas as pd
import ast
df = pd.read_excel(main_dir+'150word_label.xlsx')
number = list(df.iloc[:,0])
sequence = list(df.iloc[:,2])
sequence = [ast.literal_eval(string) for string in sequence]

In [ ]:
# -*- coding: utf-8 -*-
"""preprocess.ipynb
Automatically generated by Colaboratory.
Original file is located at
    https://colab.research.google.com/drive/1_HGxPyUOCJ3wnMfPbXl7sqWSz-jsCMGe
"""

# input: data (type: numpy array)(shape: (time, 2))
# output: data (type: numpy array)(shape: (300, 2))
import numpy as np
from scipy import signal

def preprocess(data):
  # Delete silent part
  data = data.astype(float)
  n_size = 50
  n_len = int(data.shape[0]/n_size)
  std_data = np.zeros((n_size, 2))
  for i in range(n_size):
    seg_data_x = data[i*n_len:i*n_len+n_len, 0]
    seg_data_y = data[i*n_len:i*n_len+n_len, 1]
    std_data[i, 0] = np.std(seg_data_x)
    std_data[i, 1] = np.std(seg_data_y)
  pass_threshold = 1
  pass_idx_x = np.where(std_data[:,0] >= pass_threshold)[0]
  pass_idx_y = np.where(std_data[:,1] >= pass_threshold)[0]
  if len(pass_idx_x) == 0:
    start_idx = max(0, pass_idx_y[0] - 1)
    end_idx = min(data.shape[0],pass_idx_y[-1] + 1)
  elif len(pass_idx_y) == 0:
    start_idx = max(0, pass_idx_x[0] - 1)
    end_idx = min(data.shape[0],pass_idx_x[-1] + 1)
  else:
    start_idx = max(0,min(pass_idx_x[0], pass_idx_y[0]) - 1)
    end_idx = min(data.shape[0],max(pass_idx_x[-1], pass_idx_y[-1]) + 1)
  
  # resample to 300 data points
  data = signal.resample(data[start_idx*n_len:end_idx*n_len, :], 300, axis=0)
  # scale
  data = (data - data.min(axis=0, keepdims=True))/(data.max(axis=0, keepdims=True) - data.min(axis=0, keepdims=True))
  # data = np.round(data)
  # data = (data - data.min(axis=0, keepdims=True))
  # data = data.astype(int)
  return data


# get file list for 1 patient
def get_file_list(path):
    file_list = []
    label = []
    sequence_l = []
    print(path)
    for i in os.listdir(path):
        file_list.append(i)
        l = int(i.split("_")[-1].split(".")[0])-1
        label.append(l)
        sequence_l.append(sequence[l])
    print(len(label))
    return label,file_list
    # return sequence_l,file_list

# get feature from 1 file and preprocess
def get_feature(path):
    f = []
    a = pd.read_csv(path,names=["vertical","horizontal"])
    a = np.array(a)
    a = preprocess(a)
    for j in a[:,0]:
        f.append(j)        
    for j in a[:,1]:
        f.append(j)
    return f

def evaluate(y_true, y_pred):
    f1_micro = f1_score(y_true, y_pred,average = 'micro')
    f1_macro = f1_score(y_true, y_pred,average = 'macro')
    precision_micro = precision_score(y_true, y_pred, average='micro')
    precision_macro = precision_score(y_true, y_pred, average='macro')
    recall_micro = recall_score(y_true, y_pred, average='micro')
    recall_macro = recall_score(y_true, y_pred, average='macro')
    acc = accuracy_score(y_true, y_pred)

    return f1_micro,f1_macro,precision_micro,precision_macro,recall_micro,recall_macro,acc

# Split on patient level - User-independent


> 6-fold cross-validation





In [ ]:
# self identified test_split
def my_train_test_split_user_independent(test_patient,val_patient,train_patient):
    X_test = []
    X_train = []
    X_val = []
    y_val = []
    y_train = []
    y_test = []
    for p in train_patient:
        path = str(main_dir+p+"/training/")#str("../../../../data/isolated 2/"+p+"/isolated_strokes/")
        label,file_list = get_file_list(path)
        for i in range(len(file_list)):
            file = file_list[i]
            file_label = label[i]
            feature = get_feature(str(path+file))
            X_train.append(feature)
            y_train.append(file_label)
        
    path = str(main_dir+test_patient+"/training/")#str("../../../../data/isolated 2/"+test_patient+"/isolated_strokes/")
    label,file_list = get_file_list(path)
    for i in range(len(file_list)):
        file = file_list[i]
        file_label = label[i]
        feature = get_feature(str(path+file))
        X_test.append(feature)
        y_test.append(file_label)
    
    path = str(main_dir+val_patient+"/training/")# str("../../../../data/isolated 2/"+val_patient+"/isolated_strokes/")
    label,file_list = get_file_list(path)
    for i in range(len(file_list)):
        file = file_list[i]
        file_label = label[i]
        feature = get_feature(str(path+file))
        X_val.append(feature)
        y_val.append(file_label)

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    X_val = np.array(X_val)
    y_val = np.array(y_val)
    return X_train,X_test, X_val,y_val,y_train,y_test

In [ ]:
# self identified test_split
def my_train_test_split_user_independent(test_patient,val_patient,train_patient):
  X_test = dict({})
  X_train = dict({})
  X_val = dict({})
  for i in range(150):
    X_test[i] = []
    X_train[i] = []
    X_val[i] = []
  for p in train_patient:
    path = str(main_dir+p+"/training/")
    label,file_list = get_file_list(path)
    for i in range(len(file_list)):
      file_p = file_list[i]
      file_label = label[i]
      feature = get_feature(str(path+file_p))
      if file_p.split('_')[2] in ["01", "02","03","04","05"]:#,"06","07","08","09","10"]:
        X_train[file_label].append(feature)
        
  path = str(main_dir+test_patient+"/training/")
  label,file_list = get_file_list(path)
  for i in range(len(file_list)):
    file_p = file_list[i]
    file_label = label[i]
    feature = get_feature(str(path+file_p))
    if file_p.split('_')[2] in ["01", "02","03","04","05"]:#,"06","07","08","09","10"]:
      X_test[file_label].append(feature)
    
  path = str(main_dir+val_patient+"/training/")
  label,file_list = get_file_list(path)
  for i in range(len(file_list)):
    file_p = file_list[i]
    file_label = label[i]
    feature = get_feature(str(path+file_p))
    if file_p.split('_')[2] in ["01", "02","03","04","05"]:#,"06","07","08","09","10"]:
      X_val[file_label].append(feature)

  return X_train,X_test, X_val

In [ ]:
!pip install hmmlearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 9.9 MB/s eta 0:00:00


In [ ]:
from hmmlearn import hmm
start_probability = np.array([0.52,0.48])
transition_probability = np.array([
  [0.55, 0.45],
  [0.45, 0.55]
])
emission_probability = np.array([
  [0.083, 0.085, 0.086, 0.089, 0.086, 0.084, 0.081, 0.082, 0.071, 0.081, 0.087, 0.085],
  [0.087, 0.083, 0.079, 0.088, 0.085, 0.078, 0.089, 0.08, 0.084, 0.083, 0.079, 0.085]
])
def get_stroke_hmms(n_states):
  stroke_hmms = dict()
  for i in range(12):
    model = hmm.MultinomialHMM(n_components=n_states, n_iter=20, tol=0.001, random_state=42)
    model.startprob_=start_probability
    model.transmat_=transition_probability
    model.emissionprob_=emission_probability
    stroke_hmms[i] = model
  return stroke_hmms
  
def get_word_hmms(n_states,stroke_hmms,sequence):
  num_states = n_states*len(sequence)
  word_transitions = np.zeros((num_states, num_states), dtype=float)
  word_emissions = np.zeros((12, num_states, num_states), dtype=float)
  model = hmm.MultinomialHMM(n_components=num_states, n_iter=20, tol=0.001, random_state=42)
  pointer = 0
  for i in range(len(sequence)):
    current_stroke = sequence[i]
    current_model = stroke_hmms[current_stroke]
    model = hmm.MultinomialHMM(n_components=n_states)
    start_probability = model.startprob_
    transition_probability = model.transmat_
    emission_probability = model.emissionprob_

    word_transitions[pointer: pointer + n_states, pointer:pointer + n_states] = transition_probability
    word_emissions[:, pointer: pointer + n_states, pointer:pointer + n_states] = emission_probability
    pointer += n_states

  model.transmat_=word_transitions
  model.emissionprob_=word_emissions
  return model

In [ ]:
np.min(X_train[0])

0.0

In [ ]:
len(X_train[0])

20

In [ ]:
len(X_test[0])

5

In [ ]:
test_patien_list = ["001","002","003","004","005","006"]
val_patient_list = ["002","003","004","005","006","001"]
output_pck = []
F1 = 0
N = [2,3]
output = dict()

val_pred = []
val_label = []
test_pred = []
test_label = []

for t in range(len(test_patien_list)):
    train_patient= ["001","002","003","004","005","006"]
    test_patient = test_patien_list[t]
    val_patient = val_patient_list[t]
    train_patient.remove(test_patient)
    train_patient.remove(val_patient)
    print(test_patient,val_patient,train_patient)
    # X_train,X_test, X_val,y_val,y_train,y_test = my_train_test_split_user_independent(test_patient,val_patient,train_patient)
    X_train,X_test, X_val = my_train_test_split_user_independent(test_patient,val_patient,train_patient)
    print("hey")
    clf_list = []
    for k in range(150):
      X_train_stroke = np.array(X_train[k])
      clf = hmm.GaussianHMM(n_components=3, random_state=42)
      clf.fit(X_train_stroke)
      clf_list.append(clf)
    pred = np.zeros((3,150))
    for i in range(150):
      score = np.zeros((3,150))
      for j in range(150):
        X_val_stroke = np.array(X_val[i])
        X_test_stroke = np.array(X_test[i])
        clf = clf_list[j]
        score[0,j] = clf.score(X_val_stroke)
        score[1,j] = clf.score(X_test_stroke[0].reshape(1, -1))
        score[2,j] = clf.score(X_test_stroke[1].reshape(1, -1))
      pred[:,i] = np.argmax(score,axis=1)
    val_pred.extend(list(pred[0,:]))
    val_label.extend(list(np.arange(150)))
    test_pred.extend(list(pred[1,:]))
    test_pred.extend(list(pred[2,:]))
    test_label.extend(list(np.arange(150)))
    test_label.extend(list(np.arange(150)))

001 002 ['003', '004', '005', '006']
drive/MyDrive/23Spring-MLMA/continuous1/003/training/
756
drive/MyDrive/23Spring-MLMA/continuous1/004/training/
758
drive/MyDrive/23Spring-MLMA/continuous1/005/training/
761
drive/MyDrive/23Spring-MLMA/continuous1/006/training/
761
drive/MyDrive/23Spring-MLMA/continuous1/001/training/
750
drive/MyDrive/23Spring-MLMA/continuous1/002/training/
750
hey
002 003 ['001', '004', '005', '006']
drive/MyDrive/23Spring-MLMA/continuous1/001/training/
750
drive/MyDrive/23Spring-MLMA/continuous1/004/training/
758
drive/MyDrive/23Spring-MLMA/continuous1/005/training/
761
drive/MyDrive/23Spring-MLMA/continuous1/006/training/
761
drive/MyDrive/23Spring-MLMA/continuous1/002/training/
750
drive/MyDrive/23Spring-MLMA/continuous1/003/training/
756
hey
003 004 ['001', '002', '005', '006']
drive/MyDrive/23Spring-MLMA/continuous1/001/training/
750
drive/MyDrive/23Spring-MLMA/continuous1/002/training/
750
drive/MyDrive/23Spring-MLMA/continuous1/005/training/
761
drive/MyDri

005 006 ['001', '002', '003', '004']
drive/MyDrive/23Spring-MLMA/continuous1/001/training/
750
drive/MyDrive/23Spring-MLMA/continuous1/002/training/
750
drive/MyDrive/23Spring-MLMA/continuous1/003/training/
756
drive/MyDrive/23Spring-MLMA/continuous1/004/training/
758
drive/MyDrive/23Spring-MLMA/continuous1/005/training/
761
drive/MyDrive/23Spring-MLMA/continuous1/006/training/
761


hey
006 001 ['002', '003', '004', '005']
drive/MyDrive/23Spring-MLMA/continuous1/002/training/
750
drive/MyDrive/23Spring-MLMA/continuous1/003/training/
756
drive/MyDrive/23Spring-MLMA/continuous1/004/training/
758
drive/MyDrive/23Spring-MLMA/continuous1/005/training/
761
drive/MyDrive/23Spring-MLMA/continuous1/006/training/
761
drive/MyDrive/23Spring-MLMA/continuous1/001/training/
750


hey


In [ ]:
len(val_label)

900

In [ ]:
val_pred[2]

45.0

In [ ]:
val_f1_micro,val_f1_macro,val_precision_micro,val_precision_macro,val_recall_micro,val_recall_macro,val_acc = evaluate(val_label,val_pred)
test_f1_micro,test_f1_macro,test_precision_micro,test_precision_macro,test_recall_micro,test_recall_macro,test_acc = evaluate(test_label,test_pred)
print("validation f1 micro:", val_f1_micro)
print("validation f1 macro:", val_f1_macro)
print("validation precision micro", val_precision_micro)
print("validation precision macro", val_precision_macro)
print("validation recall micro", val_recall_micro)
print("validation recall macro", val_recall_macro)
print("validation accuracy", val_acc)
print("test f1 micro:", test_f1_micro)
print("test f1 macro:", test_f1_macro)
print("test precision micro", test_precision_micro)
print("test precision macro", test_precision_macro)
print("test recall micro", test_recall_micro)
print("test recall macro", test_recall_macro)
print("test accuracy", test_acc)

validation f1 micro: 0.04777777777777778
validation f1 macro: 0.04288299831723619
validation precision micro 0.04777777777777778
validation precision macro 0.06852352706074653
validation recall micro 0.04777777777777778
validation recall macro 0.04777777777777777
validation accuracy 0.04777777777777778
test f1 micro: 0.05388888888888889
test f1 macro: 0.04978889601755972
test precision micro 0.05388888888888889
test precision macro 0.09833876518179271
test recall micro 0.05388888888888889
test recall macro 0.05388888888888888
test accuracy 0.05388888888888889


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
